In [ ]:
'''
import your life
'''
import numpy as np
import io
import datetime as date
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import re

def extract_numbers(df):
    '''
    this function is to extract the division and device name of the intersection
    df: the data frame
    '''
    # Function to extract numbers from a string
    def extract_number(text):
        if isinstance(text, str):  # Check if text is a string
            match = re.search(r'(\d{2})-(\d{4})', text)  # Match pattern 'XX-XXXX' looking for a device code
            if match:
                return match.group(1), match.group(0)
        return None, None  # Return None for both if pattern not found or not a string

    # Apply the function to create new columns
    extracted_data = df['Device'].apply(lambda x: extract_number(x))
    df['Division'], df['Device#'] = zip(*extracted_data)

    # Check column 'Intersection' if not found in 'Device'
    mask = df['Division'].isnull()  # Mask for rows where 'Division' is NaN
    df.loc[mask, 'Division'], df.loc[mask, 'Device#'] = zip(*df.loc[mask, 'Intersection'].apply(lambda x: extract_number(x)))

    def extract_numbers_from_D04(text):  # Function to extract division from column 'Group'
        if isinstance(text, str):  # Check if text is a string
            match = re.search(r'D(\d+)', text)
            if match:
                return match.group(1)
        return None  # Return None if pattern not found or not a string

    df['ColumnD04_numbers'] = df['Group'].apply(extract_numbers_from_D04)
    df['Division'] = df.apply(lambda row: row['Division'] if pd.notnull(row['Division']) else extract_numbers_from_D04(row['Group']), axis=1)
    df.drop(columns=['ColumnD04_numbers'], inplace=True)  # Drop the temporary column

    return df


In [ ]:
def calculate_time_difference(df, time_column='Received'):
  '''
  to count the total time of the data
  df: the data frame
  time_column: the column with datetime
  '''
  # Sort the DataFrame by the time column
  df_sorted = df.sort_values(by=time_column)

  # Extract time data from the first and last rows
  first_time = df_sorted.iloc[0][time_column]
  last_time = df_sorted.iloc[-1][time_column]
  # Calculate the difference in time
  time_difference = last_time - first_time

  # Convert the time difference to total seconds
  total_seconds = time_difference.total_seconds()

  return total_seconds


In [ ]:
#Reading in the csv file: (note that this does not fix the problem of columns geting cut off) hoping that a different version of python might fix the problem. If not it may be a problem in how some items are quoted
csv_file="Alarm_Report-2024-07-15_15-47-14.csv"
import csv

with open(csv_file, 'r', encoding='utf-8') as file:
  reader = csv.reader(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  data = [row for row in reader]

In [ ]:
df = pd.DataFrame(data)
df.columns = df.iloc[0] #adding the headers
df = df[1:]

In [ ]:
df

In [ ]:
df.columns = ['Received', 'Cleared', 'Severity', 'Name', 'Index', 'Device', 'Intersection', 'Group', 'Description', 'Active', 'Acknowledged', 'Comment', 'col1', 'col2', 'col3', 'col4'] # Set the new column names

In [ ]:
#rotating columns around based on patterns in the code. Note that I am writing over some data because I only need columns 'Name' 'Active', 'Acknowledged' 'Received', 'Cleared' and 'Device' with 'Intersection', 'Group', 'Description', 'Severity' 'Comment', being helpful.
box=[]
for c, row in df.iterrows():
  if pd.notna(row['col2']): #if the data has entered into col2
    if row['Name']==" Time Drift": #if time Drift
      df.at[c, 'Group'] = df.at[c, 'Active']
      df.at[c,'Description'] =df.at[c,'col1']
      df.at[c, 'Active'] =df.at[c,'col2']
      df.at[c,'Acknowledged']=df.at[c,'col3']
      df.at[c,'Comment'] = df.at[c,'col4']
      box.append(df.loc[c])
    elif row['Name']==" Database Upload Failed": #if Database Upload Failed
      if pd.notna(row['col4']): #if in col 4
        df.at[c, 'Group'] = df.at[c, 'Active']
        df.at[c,'Description'] =df.at[c,'Comment']
        df.at[c, 'Active'] =df.at[c,'col2']
        df.at[c,'Acknowledged']=df.at[c,'col3']
        box.append(df.loc[c])
      else:
        df.at[c, 'Group'] = df.at[c, 'Active']
        df.at[c,'Description'] =df.at[c,'Comment']
        df.at[c, 'Active'] =df.at[c,'col1']
        df.at[c, 'Acknowledged'] =df.at[c,'col2']
        df.at[c,'Comment'] = df.at[c,'col3']
        box.append(df.loc[c])
    else: #if in col 2 but not either of the above, note likely missing some other variance of how the data might split. If it gets fixed at the source that would be better!
      df.at[c, 'Group'] = df.at[c, 'Active']
      df.at[c,'Description'] =df.at[c,'Acknowledged']
      df.at[c,'Comment'] = df.at[c,'col3']
      df.at[c, 'Active'] =df.at[c,'col1']
      df.at[c,'Acknowledged']=df.at[c,'col2']
      box.append(df.loc[c])
  elif row['Name']==" Time Drift": #this is what the time Drift
    df.at[c, 'Description']=df.at[c,'Active']
    df.at[c,'Active']=df.at[c,'Acknowledged']
    df.at[c,'Acknowledged']=df.at[c,'Comment']
    df.at[c,'Comment'] = df.at[c,'col1']
    box.append(df.loc[c])
  elif row['Name']==" Database Upload Failed" and pd.notna(row['col1']): #this is what the Database Upload Failed
    df.at[c, 'Description']=df.at[c,'Active']
    df.at[c,'Active']=df.at[c,'Acknowledged']
    df.at[c, 'Acknowledged'] =df.at[c,'Comment']
    df.at[c,'Comment'] = df.at[c,'col1']
    box.append(df.loc[c])
  else:
    box.append(df.loc[c]) #normal (probably)
df1 = pd.DataFrame(box)

In [ ]:
df1

In [ ]:
df1['Active'] = df1['Active'].map({' true':True,' false':False,' ':False}) #forcing active and acknowledge to be Boolean
df1['Acknowledged'] = df1['Acknowledged'].map({' true':True,' false':False,' ':False})
pd.api.types.is_bool_dtype(df1['Acknowledged'])

In [ ]:
df2=df1.drop(['col1','col2','col3','col4'],axis=1) #dropping unnamedd columns

In [ ]:
df2

In [ ]:
df2['Received'] = pd.to_datetime(df2['Received'], format="%m/%d/%Y %I:%M:%S %p") #Converting to date time and converting back to UTC
df2['Received'] = df2['Received'].dt.tz_localize('Etc/GMT-2')
df2['Received'] = df2['Received'].dt.tz_convert('UTC')
df2['Cleared'] = pd.to_datetime(df2['Cleared'], format= " %m/%d/%Y %I:%M:%S %p", errors='coerce')
df2['Cleared'] = df2['Cleared'].dt.tz_localize('Etc/GMT-2')
df2['Cleared'] = df2['Cleared'].dt.tz_convert('UTC')

In [ ]:
df2['Duration'] = df2['Cleared'] - df2['Received']

In [ ]:
df2 = extract_numbers(df2) #calling the function to extract numbers to get the division and device name of the intersection

In [ ]:
df2

In [ ]:
#getting ready to sort that is row comm failure
tohist=[]
for c, row in df2.iterrows():
  if 'Device#' in row.index:
    tohist.append(df2.loc[c, ['Device#','Duration','Name']])
tohist = pd.DataFrame(tohist)

In [ ]:
tohist1 = {}
# Initialize an empty dictionary to store the transformed data
for name in tohist['Device#'].unique():
    # Select durations where comm failure
    total_duration = tohist.loc[(tohist['Device#'] == name) & (tohist['Name'] == ' Comm Failure'), 'Duration'].sum() #rotating to device name and summing the total duration of comm failure

    # Create a new key with the name and corresponding durations
    tohist1[name] = total_duration
tohist2 = pd.DataFrame(list(tohist1.items()), columns=['Device#', 'Total_Duration'])

In [ ]:
tohist2['Division'] = tohist2['Device#'].str.extract(r'^(\d{2})') #add back in the division

In [ ]:
tohist2['Total_Duration'] = pd.to_timedelta(tohist2['Total_Duration'], errors='coerce') #convert to time delta

In [ ]:
tohist2['percentage_of_time'] = tohist2['Total_Duration'].dt.total_seconds() / calculate_time_difference(df2) * 100 #averge over the whole data frame
tohist2 = tohist2.drop(tohist2[tohist2['Division'].isna()].index)

In [ ]:
tohist2

In [ ]:
df2.to_csv('Alarm Report.csv', sep=',') #write up the initial dataframe

In [ ]:
tohist2.to_csv('Comm Failure.csv', sep=',') #write the up time